In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

In [9]:
# Load data
X = pd.read_csv(r'E:\4th sem\IOBS-2\project\Non Coding RNA classification\Cleaned_data\EIIP\0-100_features.csv')
y = pd.read_csv(r"E:\4th sem\IOBS-2\project\Non Coding RNA classification\Cleaned_data\EIIP\0-100_class.csv")

# Encode target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
# Define CNN model
def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(32, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))  # Reduce to 64 features
    model.add(Dense(50, activation='relu'))  # Further reduce to less than 100 features
    return model

# Create CNN model
input_shape = (X_train.shape[1], 1)  # Assuming X_train is a DataFrame with features and 1D convolution
num_classes = len(label_encoder.classes_)
cnn_model = create_cnn_model(input_shape, num_classes)


In [11]:
# Compile CNN model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train CNN model
cnn_model.fit(X_train.values.reshape(-1, X_train.shape[1], 1), y_train, epochs=10, batch_size=32, verbose=1)

# Get the flattened output for feature extraction
cnn_features_train = cnn_model.predict(X_train.values.reshape(-1, X_train.shape[1], 1))
cnn_features_test = cnn_model.predict(X_test.values.reshape(-1, X_test.shape[1], 1))



Epoch 1/10


37/37 [==============================] - 2s 7ms/step - loss: 4.0820 - accuracy: 0.2627
Epoch 2/10
37/37 [==============================] - 0s 6ms/step - loss: 3.5372 - accuracy: 0.3497
Epoch 3/10
37/37 [==============================] - 0s 5ms/step - loss: 3.3461 - accuracy: 0.3575
Epoch 4/10
37/37 [==============================] - 0s 5ms/step - loss: 3.2607 - accuracy: 0.3988
Epoch 5/10
37/37 [==============================] - 0s 5ms/step - loss: 3.1862 - accuracy: 0.3669
Epoch 6/10
37/37 [==============================] - 0s 6ms/step - loss: 3.1662 - accuracy: 0.4065
Epoch 7/10
37/37 [==============================] - 0s 5ms/step - loss: 6.9805 - accuracy: 0.3385
Epoch 8/10
37/37 [==============================] - 0s 6ms/step - loss: 4.3757 - accuracy: 0.1042
Epoch 9/10
37/37 [==============================] - 0s 6ms/step - loss: 3.9138 - accuracy: 0.0112
Epoch 10/10
10/10 [==============================] - 0s 3ms/step


In [12]:
# Train LGBMClassifier on the extracted features
lgbm_classifier = LGBMClassifier()
lgbm_classifier.fit(cnn_features_train, y_train)

# Evaluate LGBMClassifier
accuracy = lgbm_classifier.score(cnn_features_test, y_test)
print("Accuracy of LGBMClassifier:", accuracy)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 1161, number of used features: 0
[LightGBM] [Info] Start training from score -4.572130
[LightGBM] [Info] Start training from score -5.958425
[LightGBM] [Info] Start training from score -1.714703
[LightGBM] [Info] Start training from score -4.977595
[LightGBM] [Info] Start training from score -5.111127
[LightGBM] [Info] Start training from score -1.597451
[LightGBM] [Info] Start training from score -3.049704
[LightGBM] [Info] Start training from score -1.988133
[LightGBM] [Info] Start training from score -2.996594
[LightGBM] [Info] Start training from score -2.979500
[LightGBM] [Info] Start training from score -4.166665
[LightGBM] [Info] Start training from score -1.233991
[LightGBM] [

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

In [14]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [18]:
# Calculate sequence length
sequence_length = X_train.shape[1]
num_classes = len(np.unique(y_train))

sequence_length

100

In [19]:
# Define and train more complex neural network architecture (e.g., Bidirectional LSTM)
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(sequence_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [20]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/50


ValueError: in user code:

    File "c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1209, in compute_loss
        return self.compiled_loss(
    F

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb

# Load data
X = pd.read_csv(r'E:\4th sem\IOBS-2\project\Non Coding RNA classification\Cleaned_data\EIIP\0-100_features.csv')
y = pd.read_csv(r"E:\4th sem\IOBS-2\project\Non Coding RNA classification\Cleaned_data\EIIP\0-100_class.csv")

# Preprocess data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define LightGBM model
lgb_model = lgb.LGBMClassifier()

# Train the model
lgb_model.fit(X_train, y_train)

# Evaluate the model
train_accuracy = lgb_model.score(X_train, y_train)
test_accuracy = lgb_model.score(X_test, y_test)

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)


c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000799 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 495
[LightGBM] [Info] Number of data points in the train set: 1161, number of used features: 99
[LightGBM] [Info] Start training from score -4.572130
[LightGBM] [Info] Start training from score -5.958425
[LightGBM] [Info] Start training from score -1.714703
[LightGBM] [Info] Start training from score -4.977595
[LightGBM] [Info] Start training from score -5.111127
[LightGBM] [Info] Start training from score -1.597451
[LightGBM] [Info] Start training from score -3.049704
[LightGBM] [Info] Start training from score -1.988133
[LightGBM] [Info] Start training from score -2.996594
[LightGBM] [Info] Start training from score -2.979500
[LightGBM] [Info] Start training from score -4.166665
[LightGBM] [Info] Start training from score -1.233991
[

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score

In [23]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
num_classes = len(label_encoder.classes_)

# Convert labels to one-hot encoding
y_train_one_hot = to_categorical(y_train_encoded, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test_encoded, num_classes=num_classes)


c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [26]:
# Define LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], 1)))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [28]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train_one_hot, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/50
15/15 [==============================] - 2s 128ms/step - loss: 1.9079 - accuracy: 0.2899 - val_loss: 1.8862 - val_accuracy: 0.2961
Epoch 2/50
15/15 [==============================] - 2s 118ms/step - loss: 1.9060 - accuracy: 0.2899 - val_loss: 1.8863 - val_accuracy: 0.2961
Epoch 3/50
15/15 [==============================] - 2s 125ms/step - loss: 1.9069 - accuracy: 0.2899 - val_loss: 1.8850 - val_accuracy: 0.2961
Epoch 4/50
15/15 [==============================] - 2s 127ms/step - loss: 1.9069 - accuracy: 0.2899 - val_loss: 1.8837 - val_accuracy: 0.2961
Epoch 5/50
15/15 [==============================] - 2s 119ms/step - loss: 1.9069 - accuracy: 0.2899 - val_loss: 1.8830 - val_accuracy: 0.2961
Epoch 6/50
15/15 [==============================] - 2s 118ms/step - loss: 1.9062 - accuracy: 0.2899 - val_loss: 1.8817 - val_accuracy: 0.2961
Epoch 7/50
15/15 [==============================] - 2s 119ms/step - loss: 1.9069 - accuracy: 0.2899 - val_loss: 1.8821 - val_accuracy: 0.2961
Epoch 

In [29]:
# Define the model
sequence_length = X_train.shape[1]
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(sequence_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [30]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train_one_hot, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test_one_hot)
print("Accuracy:", accuracy)

Epoch 1/50
15/15 [==============================] - 6s 122ms/step - loss: 2.2274 - accuracy: 0.2651 - val_loss: 1.9331 - val_accuracy: 0.2961
Epoch 2/50
15/15 [==============================] - 1s 38ms/step - loss: 1.9409 - accuracy: 0.2985 - val_loss: 1.8653 - val_accuracy: 0.2961
Epoch 3/50
15/15 [==============================] - 1s 35ms/step - loss: 1.9265 - accuracy: 0.2823 - val_loss: 1.8601 - val_accuracy: 0.2961
Epoch 4/50
15/15 [==============================] - 1s 37ms/step - loss: 1.8879 - accuracy: 0.3082 - val_loss: 1.8045 - val_accuracy: 0.3391
Epoch 5/50
15/15 [==============================] - 1s 40ms/step - loss: 1.8477 - accuracy: 0.3254 - val_loss: 1.7277 - val_accuracy: 0.3219
Epoch 6/50
15/15 [==============================] - 1s 35ms/step - loss: 1.7728 - accuracy: 0.3470 - val_loss: 1.6222 - val_accuracy: 0.3519
Epoch 7/50
15/15 [==============================] - 1s 35ms/step - loss: 1.7002 - accuracy: 0.3879 - val_loss: 1.6221 - val_accuracy: 0.3777
Epoch 8/50
1